In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
from fastai import *
from fastai.vision import *
import os

In [ ]:
path = Path("../input")

In [ ]:
tfms = get_transforms(do_flip=True, flip_vert=True, max_rotate=10.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75)

In [ ]:
np.random.seed(42)
src = (ImageList.from_csv(path, 'train.csv', folder='train/train')
      .split_by_rand_pct(0.2)
      .label_from_df())

In [ ]:
data = (src.transform(tfms, size=128)
       .databunch()
       .normalize(imagenet_stats))

In [ ]:
#data.show_batch(rows=3, fig_size=(7,6))

In [ ]:
arch = models.resnet50

In [ ]:
learn = cnn_learner(data, arch, metrics=accuracy, model_dir="/tmp/model/")

In [ ]:
#learn.lr_find()
#learn.recorder.plot()

In [ ]:
lr = 3e-04
learn.fit_one_cycle(7, slice(lr))

In [ ]:
learn.unfreeze()

In [ ]:
#learn.lr_find()
#learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(7, slice(1e-5, lr/5))

In [ ]:
test = ImageList.from_csv(path, 'sample_submission.csv', folder='test/test')

In [ ]:
#learn.save('stage-2-rn50')

In [ ]:
data.add_test(test)

In [ ]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
test_df = pd.read_csv("../input/sample_submission.csv")

In [ ]:
test_df.has_cactus = preds.numpy()[:, 0]

In [ ]:
test_df.to_csv('submission.csv', index=False)